Imports

In [114]:
import torchaudio
import torch
import torch.nn as nn
import torch.nn.functional as F
import librosa
import os
import numpy as np
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

Function for spliting the data into train and test set + preparing for AudioDataset class

In [125]:
def getData(audio_file_path, train = True):

    audio_waveform_list = []
    labels_list = []

    for dir in os.listdir(audio_file_path):
        for file in os.listdir(os.path.join(audio_file_path, dir)):
            if file.endswith('.wav'):
                waveform, _ = torchaudio.load(os.path.join(audio_file_path, dir, file))
                audio_waveform_list.append(waveform)
                labels_list.append(dir)


    train_waveform_list, test_waveform_list = train_test_split(audio_waveform_list, test_size=0.25, shuffle=False)
    train_labels_list, test_lables_list = train_test_split(labels_list, test_size=0.25, shuffle=False)

    if train:
        return train_waveform_list, train_labels_list
    else:
        return test_waveform_list, test_lables_list



Creating Class to load the audio files

In [126]:
class AudioDataset():
    def __init__(self, audio_file_path, train):
        self.audio_file_path = audio_file_path

        self.audio_waveform_list, self.labels_list = getData(audio_file_path, train)

    def __len__(self):
        return len(self.audio_waveform_list)
    
    def __getitem__(self, idx):
        label_encoding = {'bird': 0, 'cat': 1, 'dog': 2}
        waveform = self.audio_waveform_list[idx]
        label = self.labels_list[idx]

        return waveform, label_encoding[label]

    
def collate_fn(batch):
    waveforms, labels = zip(*batch)
    waveforms = [pad_or_truncate(w, 1600) for w in waveforms]
    waveforms = torch.stack(waveforms)
    labels = torch.tensor(labels)
    return waveforms, labels

def pad_or_truncate(waveform, target_length=16000):
    channels, samples = waveform.size()
    
    if samples > target_length:
        return waveform[:, :target_length]
    elif samples < target_length:
        pad_size = target_length - samples
        padding = torch.zeros(channels, pad_size)
        return torch.cat([waveform, padding], dim=1)
    else:
        return waveform



Loading the audio files and creating a dataset

In [127]:
audio_dataset = AudioDataset(audio_file_path='/Users/florianhaglsperger/Desktop/Coding/audioClassification/Animals', train=True)
train_laoder = DataLoader(audio_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

for batch in train_laoder:
    waveforms, labels = batch
    print(waveforms.shape)
    print(labels)
    break

torch.Size([2, 1, 1600])
tensor([2, 1])


/Users/florianhaglsperger/Desktop/Coding/.venv/lib/python3.13/site-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(


Create the model

In [ ]:
class AudioClassifier(nn.Module):
    def __init__(self, num_classes=3):
        super(AudioClassifier, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 128), nn.ReLU(),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x
    
model = AudioClassifier(num_classes=3)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

num_epochs = 50

for epoch in range(num_epochs):
    for i, (waveforms, labels) in enumerate(train_laoder):
        
        spectrograms = []
        for waveform in waveforms:
            spectrogram = librosa.feature.melspectrogram(y=waveform.numpy().squeeze(), sr=22050, n_mels=64)
            spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
            spectrograms.append(torch.tensor(spectrogram).unsqueeze(0))


        spectrograms = pad_sequence(spectrograms, batch_first=True)

        outputs = model(spectrograms)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        

/Users/florianhaglsperger/Desktop/Coding/.venv/lib/python3.13/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1600
  warnings.warn(


Epoch [1/50], Loss: 0.3784
Epoch [2/50], Loss: 1.0703
Epoch [3/50], Loss: 1.5953
Epoch [4/50], Loss: 0.7666
Epoch [5/50], Loss: 0.5928
Epoch [6/50], Loss: 0.8081
Epoch [7/50], Loss: 1.4122
Epoch [8/50], Loss: 0.9033
Epoch [9/50], Loss: 0.7096
Epoch [10/50], Loss: 0.7342
Epoch [11/50], Loss: 0.3953
Epoch [12/50], Loss: 0.7067
Epoch [13/50], Loss: 0.2059
Epoch [14/50], Loss: 0.4027
Epoch [15/50], Loss: 0.8822
Epoch [16/50], Loss: 0.8069
Epoch [17/50], Loss: 0.3696
Epoch [18/50], Loss: 0.5955
Epoch [19/50], Loss: 1.4322
Epoch [20/50], Loss: 0.8425
Epoch [21/50], Loss: 0.9118
Epoch [22/50], Loss: 0.1475
Epoch [23/50], Loss: 0.6440
Epoch [24/50], Loss: 0.3146
Epoch [25/50], Loss: 0.0595
Epoch [26/50], Loss: 0.9805
Epoch [27/50], Loss: 0.5443
Epoch [28/50], Loss: 0.6197
Epoch [29/50], Loss: 0.7026
Epoch [30/50], Loss: 0.2019
Epoch [31/50], Loss: 0.2509
Epoch [32/50], Loss: 0.9303
Epoch [33/50], Loss: 0.1733
Epoch [34/50], Loss: 0.3278
Epoch [35/50], Loss: 1.0176
Epoch [36/50], Loss: 0.5786
E

Testing the model on new data

In [130]:
dataset = AudioDataset(audio_file_path='/Users/florianhaglsperger/Desktop/Coding/audioClassification/Animals', train=False)
test_loader = DataLoader(audio_dataset, batch_size=2, shuffle=False, collate_fn=collate_fn)

model.eval()

with torch.no_grad():
    for i, (waveforms, labels) in enumerate(test_loader):
        spectrograms = []
        for waveform in waveforms:
            spectrogram = librosa.feature.melspectrogram(y=waveform.numpy().squeeze(), sr=22050, n_mels=64)
            spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
            spectrograms.append(torch.tensor(spectrogram).unsqueeze(0))


        spectrograms = pad_sequence(spectrograms, batch_first=True)

        outputs = model(spectrograms)
        loss = criterion(outputs, labels)
    
    

/Users/florianhaglsperger/Desktop/Coding/.venv/lib/python3.13/site-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/Users/florianhaglsperger/Desktop/Coding/.venv/lib/python3.13/site-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=2048 is too large for input signal of length=1600
  warnings.warn(
